
# This is a collaboratory, where you can find the code of the final project (Project 4) of the course : **Computer Vision** 

In [ ]:
# Mount google drive :

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Unzip the dataset and save it to the path "/content/imagedb_btsd" :
import os
import zipfile

local_zip = '/content/drive/MyDrive/cv_proj_4_dataset/imagedb_btsd.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/imagedb_btsd')
zip_ref.close()

In [ ]:
# Select the folder "imagedb" as the training folder
# and the folder "imagedb_test" as the testing folder :

base_dir = '/content/imagedb_btsd'

train_dir = os.path.join(base_dir, 'imagedb')
test_dir = os.path.join(base_dir, 'imagedb_test')

Using transfer learning :

In [ ]:
from keras.applications import vgg16
#Load the VGG model
vgg_conv = vgg16.VGG16(weights='imagenet', 
                 include_top=False, 
                 input_shape=(256, 256, 3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# vgg_conv2 = vgg16.VGG16(weights='imagenet', 
#                  include_top=True)

# for layer in vgg_conv2.layers:
#     print(layer, layer.trainable)

# from keras import models

# model2 = models.Sequential()
 
# # Add the vgg convolutional base model
# model2.add(vgg_conv2)

# model2.summary()
 
# # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f5273790c10> False
<keras.layers.convolutional.Conv2D object at 0x7f526a3983d0> False
<keras.layers.convolutional.Conv2D object at 0x7f5267bbd110> False
<keras.layers.pooling.MaxPooling2D object at 0x7f52602d3290> False
<keras.layers.convolutional.Conv2D object at 0x7f52602d5550> False
<keras.layers.convolutional.Conv2D object at 0x7f52602de1d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f52602d5190> False
<keras.layers.convolutional.Conv2D object at 0x7f5267b59550> False
<keras.layers.convolutional.Conv2D object at 0x7f52602e6390> False
<keras.layers.convolutional.Conv2D object at 0x7f526846d250> False
<keras.layers.pooling.MaxPooling2D object at 0x7f52602d3b90> False
<keras.layers.convolutional.Conv2D object at 0x7f52602f0310> False
<keras.layers.convolutional.Conv2D object at 0x7f52602fa290> False
<keras.layers.convolutional.Conv2D object at 0x7f52602f4b10> False
<keras.layers.pooling.MaxPooling2D object at 0x7f52602fef10>

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(34, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 1024)              33555456  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 34)                34850     
                                                                 
Total params: 48,304,994
Trainable params: 40,669,730
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range = 90, preprocessing_function=vgg16.preprocess_input, validation_split=0.2)
#train_datagen  = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# --------------------
# Flow training images in batches using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=40,
                                                    class_mode='categorical',
                                                    # color_mode='grayscale',
                                                    target_size=(256,256),
                                                    shuffle=True,
                                                    subset='training', seed=1)     
# --------------------
# Flow validation images in batches using test_datagen generator
# --------------------
validation_generator =  train_datagen.flow_from_directory(train_dir,
                                                        batch_size=40,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(256,256),
                                                        subset='validation', seed=1) 

Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


In [ ]:
model_name = 'pretrained_VGG16_batch_40_epochs_60'

In [ ]:
# Apply callbacks :

my_callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'' + model_name + '.hdf5', save_best_only=True, verbose=1)
my_callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
my_callbacks.append(early_stop_callback)

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.adam_v2.Adam(learning_rate=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=60,
      validation_data=validation_generator,
      verbose=1,
      callbacks = my_callbacks)
 
# # Save the model
# model_name = 'pretrained_VGG16_batch_40_epochs_60'
# model.save(model_name + '.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if sys.path[0] == '':


Epoch 1/60
62/61 [==============================] - ETA: 0s - loss: 3.8761 - acc: 0.2291
Epoch 00001: val_loss improved from inf to 2.02838, saving model to pretrained_VGG16_batch_40_epochs_60.hdf5
61/61 [==============================] - 80s 969ms/step - loss: 3.8761 - acc: 0.2291 - val_loss: 2.0284 - val_acc: 0.4290
Epoch 2/60
62/61 [==============================] - ETA: 0s - loss: 1.6398 - acc: 0.5425
Epoch 00002: val_loss improved from 2.02838 to 1.14018, saving model to pretrained_VGG16_batch_40_epochs_60.hdf5
61/61 [==============================] - 46s 754ms/step - loss: 1.6398 - acc: 0.5425 - val_loss: 1.1402 - val_acc: 0.6912
Epoch 3/60
62/61 [==============================] - ETA: 0s - loss: 0.9980 - acc: 0.7106
Epoch 00003: val_loss improved from 1.14018 to 0.77712, saving model to pretrained_VGG16_batch_40_epochs_60.hdf5
61/61 [==============================] - 47s 757ms/step - loss: 0.9980 - acc: 0.7106 - val_loss: 0.7771 - val_acc: 0.7780
Epoch 4/60
62/61 [==============

In [ ]:
# Saving the model to a google drive folder :

import shutil
shutil.copy('/content/'+ model_name +'.hdf5','/content/drive/MyDrive/CV_Project_4_saved_models/'+ model_name)


'/content/drive/MyDrive/CV_Project_4_saved_models/pretrained_VGG16_batch_40_epochs_60'

In [ ]:
test_datagen  = ImageDataGenerator(rescale=1./255)

# --------------------
# Flow validation images in batches using test_datagen generator
# --------------------
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                        batch_size=10,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(256,256))
# Testing the CNN on testing data : 
loss, acc = model.evaluate(test_generator)

Found 2149 images belonging to 34 classes.
215/215 [==============================] - 39s 162ms/step - loss: 3.1641 - acc: 0.0847


In [ ]:
# # Loading the model from the google drive folder : 

# model_loaded = models.load_model('/content/drive/MyDrive/CV_Project_4_saved_models/' + model_name)
# model_loaded.summary()